In [1]:
import pandas as pd
from collections import defaultdict
import warnings
import visualize
import neat
import multiprocessing
import os
import matplotlib.pyplot as plt

import numpy as np

In [2]:
necoc = 5
generations = int(3000/necoc)
generations

600

In [3]:
def softmax(x, axis=1):
    # 计算每行的最大值
    row_max = x.max(axis=axis)
 
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

In [4]:
df = pd.read_csv('letter-recognition.data',header=None)

train_df = df[:1600]
test_df = df[16000:16000+400]

X = train_df.ix[:,1:16].to_numpy() 
y = np.array([ord(l)-ord('A') for l in train_df.ix[:,0].to_numpy()])

In [5]:
def get_winner(class1):
    
    numbers = [*range(26)] # number 2 > number 1
    number_of_classification = len(numbers)
    #number_of_sampling = 100

    list_y = y.tolist()
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

        
    class2 = []

    class1_indexes = []
    class2_indexes = []

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)

    print("number of class1:{}".format(len(class1_label)))
    print("number of class2:{}".format(len(class2_label)))

    samplesize = 1600
    x_inputs =[tuple(c) for c in X[class1_indexes].tolist()] + [tuple(c) for c in X[class2_indexes].tolist()]
    x_outputs = [tuple([c]) for c in class1_label + class2_label]
    
    
    def eval_genomes(genomes, config):
        for genome_id, genome in genomes:
            net = neat.nn.FeedForwardNetwork.create(genome, config)

            outputs = []
            for xi in x_inputs:
                output = net.activate(xi)
                outputs.append(output)


            px_outputs = softmax(np.array(outputs).reshape(samplesize, 2), axis=1)
            # the index of maximum in each line
            pred_outputs = np.argmax(px_outputs, axis = 1)
            real_outputs = np.array(x_outputs).reshape(samplesize,)

            acc = np.sum(pred_outputs == real_outputs)/samplesize

            genome.fitness = acc

    def run(config_file):
        # Load configuration.
        config = neat.Config(
            neat.DefaultGenome,
            neat.DefaultReproduction,
            neat.DefaultSpeciesSet,
            neat.DefaultStagnation,
            config_file,
        )

        # Create the population, which is the top-level object for a NEAT run.
        p = neat.Population(config)

        # add a stdout reporter to show progress in the terminal
        reporter = neat.StdOutReporter(False)
        p.add_reporter(reporter)
        stats = neat.StatisticsReporter()
        p.add_reporter(stats)
        #checkpointer = neat.Checkpointer(100)
        #p.add_reporter(checkpointer)
        # Run for up to 300 generations.
        winner = p.run(eval_genomes, generations)

        return [stats, winner]
    
    local_dir = os.getcwd()
    config_path = os.path.join(local_dir, "config-feedforward-letter2")
    [stats, winner] = run(config_path)
    
    return winner

In [10]:
import numpy as np
import random

number_codes = []
for i in range(2**(26-1), 2**26-1):
    if (i % 1000000) == 0:
            print(i)
    for j in bin(i).split('b')[1]:
        number_codes.append(int(j))
        


number_codes1 = np.array(number_codes).reshape(2**(26 - 1) - 1,26).T     


34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000
60000000
61000000
62000000
63000000
64000000
65000000
66000000
67000000


In [14]:
while True:
    slice = random.sample(range(2**(26 - 1) - 1), necoc)

    number_codes = number_codes1[:,slice].tolist()

    classes = []
    for i in range(necoc):
        code_list = np.array(number_codes).T[i]
        classes.append(np.where(code_list==1)[0].tolist())

    matrix_errors = []

    for i in range(25):
        for j in range(i+1,26):
            matrix_errors.append(necoc - np.sum(np.array(number_codes[i]) == np.array(number_codes[j])))
    min_error = np.min(matrix_errors)
    if min_error != 0:
        print(min_error)
        break

1


In [15]:
winner_list = []

for i,class1 in enumerate(classes):
    print("running:{}/5".format(i))
    winner = get_winner(class1)
    winner_list.append(winner)

running:0/5
number of class1:713
number of class2:887

 ****** Running generation 0 ****** 

Population's average fitness: 0.49651 stdev: 0.05542
Best fitness: 0.59937 - size: (2, 3) - species 2 - id 19
Average adjusted fitness: 0.115
Mean genetic distance 2.640, standard deviation 0.494
Population of 201 members in 12 species
Total extinctions: 0
Generation time: 5.172 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.51577 stdev: 0.05413
Best fitness: 0.61187 - size: (3, 5) - species 6 - id 303
Average adjusted fitness: 0.097
Mean genetic distance 2.648, standard deviation 0.520
Population of 200 members in 12 species
Total extinctions: 0
Generation time: 3.911 sec (4.541 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.52776 stdev: 0.05079
Best fitness: 0.61313 - size: (3, 6) - species 6 - id 562
Average adjusted fitness: 0.150
Mean genetic distance 2.715, standard deviation 0.536
Population of 201 members in 15 species
Total e

Population's average fitness: 0.54650 stdev: 0.06135
Best fitness: 0.64187 - size: (4, 10) - species 8 - id 1521
Average adjusted fitness: 0.196
Mean genetic distance 2.974, standard deviation 0.535
Population of 201 members in 23 species
Total extinctions: 0
Generation time: 4.743 sec (4.493 average)

 ****** Running generation 25 ****** 

Population's average fitness: 0.54332 stdev: 0.06345
Best fitness: 0.64187 - size: (4, 10) - species 8 - id 1521
Average adjusted fitness: 0.126
Mean genetic distance 3.025, standard deviation 0.544
Population of 203 members in 23 species
Total extinctions: 0
Generation time: 4.753 sec (4.547 average)

 ****** Running generation 26 ****** 

Population's average fitness: 0.54371 stdev: 0.06215
Best fitness: 0.64187 - size: (4, 10) - species 8 - id 1521
Average adjusted fitness: 0.121
Mean genetic distance 3.053, standard deviation 0.555
Population of 197 members in 22 species
Total extinctions: 0
Generation time: 4.896 sec (4.597 average)

 ****** Ru

Population's average fitness: 0.56251 stdev: 0.06924
Best fitness: 0.66250 - size: (11, 19) - species 24 - id 5888
Average adjusted fitness: 0.178
Mean genetic distance 3.184, standard deviation 0.548
Population of 195 members in 19 species
Total extinctions: 0
Generation time: 5.676 sec (5.666 average)

 ****** Running generation 49 ****** 

Population's average fitness: 0.56067 stdev: 0.06646
Best fitness: 0.66250 - size: (11, 19) - species 24 - id 5888
Average adjusted fitness: 0.162
Mean genetic distance 3.178, standard deviation 0.542
Population of 198 members in 20 species
Total extinctions: 0
Generation time: 5.753 sec (5.678 average)

 ****** Running generation 50 ****** 

Population's average fitness: 0.56024 stdev: 0.06416
Best fitness: 0.66250 - size: (11, 19) - species 24 - id 5888
Average adjusted fitness: 0.145
Mean genetic distance 3.198, standard deviation 0.556
Population of 197 members in 20 species
Total extinctions: 0
Generation time: 5.741 sec (5.711 average)

 ***


Population's average fitness: 0.56181 stdev: 0.06950
Best fitness: 0.67250 - size: (15, 31) - species 41 - id 10472
Average adjusted fitness: 0.129
Mean genetic distance 3.140, standard deviation 0.596
Population of 202 members in 16 species
Total extinctions: 0
Generation time: 6.277 sec (6.705 average)

 ****** Running generation 73 ****** 

Population's average fitness: 0.56187 stdev: 0.06773
Best fitness: 0.67250 - size: (15, 31) - species 41 - id 10472
Average adjusted fitness: 0.172
Mean genetic distance 3.117, standard deviation 0.594
Population of 202 members in 14 species
Total extinctions: 0
Generation time: 6.449 sec (6.717 average)

 ****** Running generation 74 ****** 

Population's average fitness: 0.55911 stdev: 0.06541
Best fitness: 0.67250 - size: (15, 31) - species 41 - id 10472
Average adjusted fitness: 0.164
Mean genetic distance 3.156, standard deviation 0.588
Population of 202 members in 15 species
Total extinctions: 0
Generation time: 6.612 sec (6.727 average)



Population's average fitness: 0.56708 stdev: 0.06054
Best fitness: 0.67250 - size: (15, 31) - species 41 - id 10472
Average adjusted fitness: 0.181
Mean genetic distance 3.087, standard deviation 0.542
Population of 199 members in 18 species
Total extinctions: 0
Generation time: 7.053 sec (7.183 average)

 ****** Running generation 97 ****** 

Population's average fitness: 0.57580 stdev: 0.06206
Best fitness: 0.67250 - size: (15, 31) - species 41 - id 10472
Average adjusted fitness: 0.143
Mean genetic distance 3.075, standard deviation 0.546
Population of 200 members in 18 species
Total extinctions: 0
Generation time: 7.250 sec (7.250 average)

 ****** Running generation 98 ****** 

Population's average fitness: 0.56983 stdev: 0.06359
Best fitness: 0.67250 - size: (15, 31) - species 41 - id 10472
Average adjusted fitness: 0.140
Mean genetic distance 3.071, standard deviation 0.553
Population of 203 members in 18 species
Total extinctions: 0
Generation time: 7.474 sec (7.326 average)

 

...

Population's average fitness: 0.57311 stdev: 0.06994
Best fitness: 0.72313 - size: (29, 58) - species 77 - id 60204
Average adjusted fitness: 0.117
Mean genetic distance 2.806, standard deviation 0.490
Population of 201 members in 8 species
Total extinctions: 0
Generation time: 8.876 sec (8.781 average)

 ****** Running generation 491 ****** 

Population's average fitness: 0.56856 stdev: 0.07227
Best fitness: 0.72313 - size: (29, 58) - species 77 - id 60204
Average adjusted fitness: 0.121
Mean genetic distance 2.781, standard deviation 0.500
Population of 199 members in 8 species
Total extinctions: 0
Generation time: 8.554 sec (8.807 average)

 ****** Running generation 492 ****** 

Population's average fitness: 0.57473 stdev: 0.07426
Best fitness: 0.72313 - size: (29, 58) - species 77 - id 60204
Average adjusted fitness: 0.116
Mean genetic distance 2.868, standard deviation 0.514
Population of 200 members in 9 species
Total extinctions: 0
Generation time: 8.611 sec (8.768 average)

 *

Population's average fitness: 0.56840 stdev: 0.07766
Best fitness: 0.72313 - size: (29, 58) - species 77 - id 60204
Average adjusted fitness: 0.081
Mean genetic distance 2.864, standard deviation 0.493
Population of 200 members in 11 species
Total extinctions: 0
Generation time: 8.187 sec (8.491 average)

 ****** Running generation 515 ****** 

Population's average fitness: 0.55832 stdev: 0.07384
Best fitness: 0.72313 - size: (29, 58) - species 77 - id 60204
Average adjusted fitness: 0.077
Mean genetic distance 2.914, standard deviation 0.505
Population of 200 members in 12 species
Total extinctions: 0
Generation time: 8.966 sec (8.548 average)

 ****** Running generation 516 ****** 

Population's average fitness: 0.57106 stdev: 0.07607
Best fitness: 0.72313 - size: (29, 58) - species 77 - id 60204
Average adjusted fitness: 0.075
Mean genetic distance 2.930, standard deviation 0.503
Population of 202 members in 12 species
Total extinctions: 0
Generation time: 9.102 sec (8.570 average)


Population's average fitness: 0.56726 stdev: 0.07364
Best fitness: 0.72313 - size: (29, 58) - species 77 - id 60204
Average adjusted fitness: 0.066
Mean genetic distance 2.829, standard deviation 0.456
Population of 199 members in 13 species
Total extinctions: 0
Generation time: 10.943 sec (9.527 average)

 ****** Running generation 539 ****** 

Population's average fitness: 0.57004 stdev: 0.07634
Best fitness: 0.72313 - size: (29, 58) - species 77 - id 60204
Average adjusted fitness: 0.067
Mean genetic distance 2.829, standard deviation 0.457
Population of 200 members in 13 species
Total extinctions: 0
Generation time: 9.663 sec (9.565 average)

 ****** Running generation 540 ****** 

Population's average fitness: 0.56482 stdev: 0.07468
Best fitness: 0.72313 - size: (29, 58) - species 77 - id 60204
Average adjusted fitness: 0.070
Mean genetic distance 2.838, standard deviation 0.444
Population of 202 members in 13 species
Total extinctions: 0
Generation time: 9.497 sec (9.606 average)

Population's average fitness: 0.55697 stdev: 0.06718
Best fitness: 0.72313 - size: (29, 58) - species 77 - id 60204
Average adjusted fitness: 0.078
Mean genetic distance 2.763, standard deviation 0.476
Population of 199 members in 9 species
Total extinctions: 0
Generation time: 8.644 sec (9.267 average)

 ****** Running generation 563 ****** 

Population's average fitness: 0.55650 stdev: 0.06897
Best fitness: 0.72313 - size: (29, 58) - species 77 - id 60204
Average adjusted fitness: 0.060
Mean genetic distance 2.782, standard deviation 0.456
Population of 199 members in 8 species
Total extinctions: 0
Generation time: 8.944 sec (9.156 average)

 ****** Running generation 564 ****** 

Population's average fitness: 0.55161 stdev: 0.06645
Best fitness: 0.72313 - size: (29, 58) - species 77 - id 60204
Average adjusted fitness: 0.119
Mean genetic distance 2.767, standard deviation 0.483
Population of 200 members in 8 species
Total extinctions: 0
Generation time: 8.845 sec (8.993 average)

 *

Population's average fitness: 0.56102 stdev: 0.07057
Best fitness: 0.72313 - size: (29, 58) - species 77 - id 60204
Average adjusted fitness: 0.110
Mean genetic distance 2.755, standard deviation 0.523
Population of 201 members in 8 species
Total extinctions: 0
Generation time: 9.169 sec (9.391 average)

 ****** Running generation 587 ****** 

Population's average fitness: 0.55483 stdev: 0.06616
Best fitness: 0.72313 - size: (29, 58) - species 77 - id 60204
Average adjusted fitness: 0.060
Mean genetic distance 2.752, standard deviation 0.504
Population of 200 members in 7 species
Total extinctions: 0
Generation time: 10.308 sec (9.447 average)

 ****** Running generation 588 ****** 

Population's average fitness: 0.55947 stdev: 0.06867
Best fitness: 0.72313 - size: (29, 58) - species 77 - id 60204
Average adjusted fitness: 0.074
Mean genetic distance 2.705, standard deviation 0.546
Population of 201 members in 6 species
Total extinctions: 0
Generation time: 9.852 sec (9.429 average)

 

In [16]:
testX = test_df.ix[:,1:16].to_numpy() 
testy = np.array([ord(l)-ord('A') for l in test_df.ix[:,0].to_numpy()])

local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-letter2")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

def get_pred_real(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = testX[j]
    test_y = testy[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

from random import randint
pred_value = []
error_list = []

for j in range(400):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_real(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(necoc - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )
list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
np.sum(list_P == testy)/400

0.1975

In [17]:
def get_pred_train(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = X[j]
    test_y = y[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

pred_value = []
error_list = []

for j in range(1600):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_train(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(26 - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )

list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
print("Trainset acc:{}".format(np.sum(list_P == y)/1600))
winner_fitness = []
for winner in winner_list:
    #print(winner.fitness)
    winner_fitness.append(winner.fitness)
print("Avg Base Learner:{}".format(np.mean(winner_fitness)))

Trainset acc:0.183125
Avg Base Learner:0.691


In [18]:
# program to check if there is exist a path between two vertices 
# of a graph 

from collections import defaultdict 

#This class represents a directed graph using adjacency list representation 
class Graph: 

	def __init__(self,vertices): 
		self.V= vertices #No. of vertices 
		self.graph = defaultdict(list) # default dictionary to store graph 

	# function to add an edge to graph 
	def addEdge(self,u,v): 
		self.graph[u].append(v) 
	
	# Use BFS to check path between s and d 
	def isReachable(self, s, d): 
		# Mark all the vertices as not visited 
		visited =[False]*(self.V) 

		# Create a queue for BFS 
		queue=[] 

		# Mark the source node as visited and enqueue it 
		queue.append(s) 
		visited[s] = True

		while queue: 

			#Dequeue a vertex from queue 
			n = queue.pop(0) 
			
			# If this adjacent node is the destination node, 
			# then return true 
			if n == d: 
				return True

			# Else, continue to do BFS 
			for i in self.graph[n]: 
				if visited[i] == False: 
					queue.append(i) 
					visited[i] = True
		# If BFS is complete without visited d 
		return False
    
    
def findAllPath(graph,start,end,path=[]):
    path = path +[start]
    if start == end:
        return [path]
 
    paths = [] #存储所有路径    
    for node in graph[start]:
        if node not in path:
            newpaths = findAllPath(graph,node,end,path) 
            for newpath in newpaths:
                paths.append(newpath)
    return paths

numbers = [*range(2)]
number_of_classification = len(numbers)

list_nodes_number = []
list_connection_number = []

for winner in winner_list:
    used_nodes = list(winner.nodes.keys())
    # create a dict for mapping 
    l1 = numbers + config.genome_config.input_keys + used_nodes[number_of_classification:]
    l2 = range(len(l1))
    dict_nodes = dict(zip(l1, l2))

    ### 将节点加入图
    g = Graph(len(l1))# inputs + outputs + used  
    # add connections in the graph
    for cg in winner.connections.values():
        if cg.enabled:
            g.addEdge(dict_nodes[cg.key[0]],dict_nodes[cg.key[1]])

    v = numbers
    list_connections = []
    for vi in v:
        for u in range(number_of_classification, len(config.genome_config.input_keys) + number_of_classification):
            list_connections.append(g.isReachable(u, vi))

    nodes_inputs = range(number_of_classification, number_of_classification+ 64)
    nodes_outputs = range(number_of_classification)
    nodes_mid = range(number_of_classification + 64, len(dict_nodes))

    # 找到所有输入和输出连接的路
    all_path = []
    for u in nodes_inputs:
        for v in nodes_outputs:
            path = findAllPath(g.graph, u, v)
            if path:
                all_path = all_path + path

    # 得到最长路，以及每个节点在哪个层级
    max_length = max([len(x) for x in all_path])
    nodes_tuples_list = []
    for path in all_path:
        #print(path)
        for node in path:
            nodes_tuples_list.append([node, path.index(node)])

    # 确定节点的层级（消除重复）       
    nodes_tuples_fixed_list = []
    for index in range(max_length):
        for nodes in nodes_tuples_list:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                if nodes not in nodes_tuples_fixed_list:
                    nodes_tuples_fixed_list.append(nodes)


    # 所有有连接的节点
    all_used_nodes = []
    for node in nodes_tuples_fixed_list:
        if node[0] not in all_used_nodes:
            all_used_nodes.append(node[0])

    # 确定节点的层级（最终确定）
    nodes_tuples_fixed_list_final = []

    for node in all_used_nodes:
        if node in nodes_inputs:
            nodes_tuples_fixed_list_final.append([node, 0])
        elif node in nodes_outputs:
            nodes_tuples_fixed_list_final.append([node, max_length - 1])
        else:
            list_node = []
            for nodes in nodes_tuples_fixed_list:
                if node == nodes[0]:
                    list_node.append(nodes)
            layer = 0
            for ele in list_node:
                # 删除重复项，取最大
                if ele[1] > layer:
                    layer = ele[1]
            nodes_tuples_fixed_list_final.append([node,layer])


    # 每个层级的节点合数
    length_of_layers = []
    number_of_nodes = 0
    for index in range(max_length):
        for nodes in nodes_tuples_fixed_list_final:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                number_of_nodes += 1
        length_of_layers.append(number_of_nodes)

    length_of_each_layer = []
    length_of_each_layer.append(length_of_layers[0])
    for i in range(1, max_length):
        length_of_each_layer.append(length_of_layers[i] - length_of_layers[i-1])

    # 输出每个层级的节点个数
    #print("length of each layers:", length_of_each_layer)

    # 所有端到端的路
    all_path_side2side = []
    for path in all_path:
        if len(path) == 2:
            all_path_side2side.append(path)
        else:
            for i in range(len(path)-1):
                all_path_side2side.append([path[i],path[i+1]])

    # 定义节点到绘图的字典
    dict_nodes_graph = dict()
    count_number_layer = [0] * max_length
    for nodes in nodes_tuples_fixed_list_final:
        node, layer = nodes
        dict_nodes_graph[node] = [count_number_layer[layer], layer]
        count_number_layer[layer] += 1
    list_nodes_number.append(np.sum(length_of_each_layer))
    list_connection_number.append(len(all_path_side2side))
    #print("Number of nodes:{} Number of connections:{}".format(np.sum(length_of_each_layer),len(all_path_side2side)))
print("Total nodes:{} Total connections::{}".format(np.sum(list_nodes_number), np.sum(list_connection_number)))

Total nodes:147 Total connections::3047
